In [1]:
# Minimal import path setup for local src
import sys, os
from pathlib import Path
_HERE = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
_SRC = _HERE.parent.parent  # .../src
if str(_SRC) not in sys.path:
    sys.path.insert(0, str(_SRC))

# Now we can import the function directly from the package
from negf.self_energy import surface_greens_function

# Quick checks: 1D lead with H00=0, H01=-1 should return consistent G00 for small imaginary E
import numpy as np
E_list = [1j*1e-9, 1j*1e-6]
for E in E_list:
    G00 = surface_greens_function(0, np.array([[0+0j]], dtype=complex), np.array([[-1+0j]], dtype=complex))
    print(f"E={E}: G00={G00}")

[[0.+1.e-09j]] [[0.+0.j]] [[-1.+0.j]] 1e-06 1000
E=1e-09j: G00=[[0.-0.07839992j]]
[[0.+1.e-09j]] [[0.+0.j]] [[-1.+0.j]] 1e-06 1000
E=1e-06j: G00=[[0.-0.07839992j]]


In [10]:
# Analytic 1D semi-infinite chain check vs numeric surface GF
import numpy as np

def analytic_surface_g_1d(E: complex, t: float = 1.0) -> complex:
    """Retarded surface Green's function for 1D semi-infinite chain with onsite 0 and hopping -t.
    Formula: G_s(E) = (E - sqrt(E^2 - 4 t^2)) / (2 t^2), with the retarded branch for sqrt.
    """
    # Use scimath.sqrt to allow complex branch naturally
    sqrt_term = np.lib.scimath.sqrt(E**2 - (2*t)**2)
    # Enforce retarded branch: Im(sqrt) should have same sign as Im(E) (>=0 for Im(E)>0)
    if (np.imag(E) >= 0 and np.imag(sqrt_term) < 0) or (np.imag(E) < 0 and np.imag(sqrt_term) > 0):
        sqrt_term = -sqrt_term
    return (E - sqrt_term) / (2 * t**2)

# Compare for a few energies (t=1, H01 = -t)
t = 1.0
H01 = np.array([[-t]], dtype=complex)
H10 = H01.conj().T
E_list = [1j*1e-9, 1j*1e-6, 0.5 + 1j*1e-6, -1.5 + 1j*1e-6]
for E in E_list:
    G_num = test_self_energy(E)  # returns G00 from module implementation
    G_ana = analytic_surface_g_1d(E, t)
    Sigma_num = H10 @ G_num @ H01
    Sigma_ana = (t**2) * G_ana
    print(f"E={E}")
    print("  G_num:", G_num, " G_ana:", G_ana, " |ΔG|=", np.linalg.norm(G_num - G_ana))
    print("  Sigma_num:", Sigma_num, " Sigma_ana:", Sigma_ana, " |ΔΣ|=", np.linalg.norm(Sigma_num - Sigma_ana))
    print("---")

E=1e-09j
  G_num: [[0.-0.07839992j]]  G_ana: -0.9999999995j  |ΔG|= 0.9216000759322704
  Sigma_num: [[0.-0.07839992j]]  Sigma_ana: -0.9999999995j  |ΔΣ|= 0.9216000759322704
---
E=1e-06j
  G_num: [[0.-0.99996661j]]  G_ana: -0.999999500000125j  |ΔG|= 3.2891117459410424e-05
  Sigma_num: [[0.-0.99996661j]]  Sigma_ana: -0.999999500000125j  |ΔΣ|= 3.2891117459410424e-05
---
E=(0.5+1e-06j)
  G_num: [[0.24999987-0.96824534j]]  G_ana: (0.24999987090055512-0.968245336551992j)  |ΔG|= 7.776515945702112e-16
  Sigma_num: [[0.24999987-0.96824534j]]  Sigma_ana: (0.24999987090055512-0.968245336551992j)  |ΔΣ|= 7.776515945702112e-16
---
E=(-1.5+1e-06j)
  G_num: [[-0.74999943-0.66143733j]]  G_ana: (-0.7499994330532905-0.6614373277665796j)  |ΔG|= 1.3506446028928519e-15
  Sigma_num: [[-0.74999943-0.66143733j]]  Sigma_ana: (-0.7499994330532905-0.6614373277665796j)  |ΔΣ|= 1.3506446028928519e-15
---
